In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import linearmodels as lm
import matplotlib
import math
import statsmodels.formula.api as smf

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel import compare
from matplotlib import pyplot as plt

from pandas.api.types import is_numeric_dtype

pd.set_option('display.max_columns', 500)

%matplotlib inline

In [2]:
# read the main data set
df=pd.read_csv('final_dataset_oct_24.csv',
                    encoding='utf-8')


In [5]:
# exclude if ridership is zero--missing data
df = df[df['UPT']>0]

In [6]:
# separate Bus, Rail, DemandResponsive and Other
bus = df[df['Mode']=='Bus']
rail = df[df['Mode']=='Rail']


In [7]:
# set the indices
bus=bus.set_index(['MNAME','Year'])
rail=rail.set_index(['MNAME','Year'])

In [8]:
# keep only the numeric columns -- the estimation will give an error otherwise
bus = bus.select_dtypes(include=[np.number])
rail = rail.select_dtypes(include=[np.number])

In [9]:
# create a log of all fields
for col in bus.columns:
    bus[col+'_log'] = np.log(bus[col]+1)
    
for col in rail.columns:
    rail[col+'_log'] = np.log(rail[col]+1)    

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: invalid value encountered in log
  app.launch_new_instance()
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: invalid value encountered in log


In [10]:
# check the data
bus.head()

CBSA  NEW_CBSA     UPT     VRM   UPT_ADJ  \
MNAME                  Year                                              
Abilene, TX Metro Area 2002  10180       NaN  493073  389678  493073.0   
                       2003  10180       NaN  475747  383239  475747.0   
                       2004  10180       NaN  469981  388907  469981.0   
                       2005  10180       NaN  507165  444176  507165.0   
                       2006  10180       NaN  493638  441300  493638.0   

                              VRM_ADJ  VRH_ADJ  FARE_TOTAL  FARE_per_UPT  \
MNAME                  Year                                                
Abilene, TX Metro Area 2002  389678.0  29016.0    162609.0      0.329787   
                       2003  383239.0  28597.0    173563.0      0.364822   
                       2004  388907.0  29126.0    185046.0      0.393731   
                       2005  444176.0  31147.0    213965.0      0.421884   
                       2006  441300.0  30750.0    201795.0      0.408791   

                             AREALANDPT      Tot_Pop  Tot_Instate_Pop  \
MNAME                  Year                                             
Abilene, TX Metro Area 2002   141756054  117258.2188      87723.96875   
                       2003   141756054  127098.1250      93677.12500   
                       2004   141756054  137113.5000      99787.50000   
                       2005   141756054  146427.0000     105269.00000   
                       2006   141756054  158548.0000     113266.00000   

                             Tot_Outstate_Pop  Tot_NonUSA_POP  \
MNAME                  Year                                     
Abilene, TX Metro Area 2002       23752.17188     2987.484375   
                       2003       26964.31250     2834.062500   
                       2004       30227.75000     2648.750000   
                       2005       33286.00000     2591.000000   
                       2006       37165.00000     2023.000000   

                             Total_Median_Income_Individual  \
MNAME                  Year                                   
Abilene, TX Metro Area 2002                     19756.67188   
                       2003                     19941.31250   
                       2004                     20132.75000   
                       2005                     20297.00000   
                       2006                     20570.00000   

                             Native_Instate_Med_Inc_Indiv  \
MNAME                  Year                                 
Abilene, TX Metro Area 2002                   22848.10938   
                       2003                   21972.56250   
                       2004                   21085.75000   
                       2005                   20244.00000   
                       2006                   19222.00000   

                             Native_Outstate_Med_Inc_Indiv  Total_Pop_Poverty  \
MNAME                  Year                                                     
Abilene, TX Metro Area 2002                    17091.51563          147631.94   
                       2003                    18472.93750          147198.25   
                       2004                    19808.25000          146801.00   
                       2005                    21328.00000          146258.00   
                       2006                    22110.00000          146298.00   

                             Pop_Below100_Poverty  Pop_Below150_Poverty  \
MNAME                  Year                                               
Abilene, TX Metro Area 2002                 22.30                 12.89   
                       2003                 20.72                 12.86   
                       2004                 19.13                 12.88   
                       2005                 17.60                 12.70   
                       2006                 15.80                 13.30   

                             Pop_Above150_Pove

In [11]:
rail.head()

CBSA  NEW_CBSA      UPT      VRM    UPT_ADJ  \
MNAME                      Year                                                 
Albuquerque, NM Metro Area 2009  10740       NaN  1351087   813885  1351087.0   
                           2010  10740       NaN  1165687  1415143  1165687.0   
                           2011  10740       NaN  1242174  1370680  1242174.0   
                           2012  10740       NaN  1129303  1424748  1129303.0   
                           2013  10740       NaN  1082588  1388562  1082588.0   

                                   VRM_ADJ  VRH_ADJ  FARE_TOTAL  FARE_per_UPT  \
MNAME                      Year                                                 
Albuquerque, NM Metro Area 2009   813885.0  20107.0   2669729.0      1.975986   
                           2010  1415143.0  37249.0   2846221.0      2.441668   
                           2011  1370680.0  36676.0   2856263.0      2.299407   
                           2012  1424748.0  36870.0   2616485.0      2.316903   
                           2013  1388562.0  36503.0   3002928.0      2.773842   

                                 AREALANDPT   Tot_Pop  Tot_Instate_Pop  \
MNAME                      Year                                          
Albuquerque, NM Metro Area 2009   648969769  856216.0         432274.0   
                           2010   648969769  892014.0         459673.0   
                           2011   648969769  899149.0         466738.0   
                           2012   648969769  902794.0         484743.0   
                           2013   648969769  901932.0         472342.0   

                                 Tot_Outstate_Pop  Tot_NonUSA_POP  \
MNAME                      Year                                     
Albuquerque, NM Metro Area 2009          329891.0         11065.0   
                           2010          332367.0          9139.0   
                           2011          336918.0          9294.0   
                           2012          320910.0         12335.0   
                           2013          328386.0         11036.0   

                                 Total_Median_Income_Individual  \
MNAME                      Year                                   
Albuquerque, NM Metro Area 2009                         25253.0   
                           2010                         25891.0   
                           2011                         24924.0   
                           2012                         25616.0   
                           2013                         25717.0   

                                 Native_Instate_Med_Inc_Indiv  \
MNAME                      Year                                 
Albuquerque, NM Metro Area 2009                       23216.0   
                           2010                       23628.0   
                           2011                       21970.0   
                           2012                       22332.0   
                           2013                       22709.0   

                                 Native_Outstate_Med_Inc_Indiv  \
MNAME                      Year                                  
Albuquerque, NM Metro Area 2009                        30017.0   
                           2010                        31267.0   
                           2011                        30564.0   
                           2012                        31553.0   
                           2013                        31723.0   

                                 Total_Pop_Poverty  Pop_Below100_Poverty  \
MNAME                      Year                                            
Albuquerque, NM Metro Area 2009           846650.0                  15.9   
                           2010           881703.0                   NaN   
                           2011           887261.0                   NaN   
                           2012           887938.0                   NaN   
                           2013           890054.0

In [16]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.1866
Estimator:                   PanelOLS   R-squared (Between):              0.7625
No. Observations:                4504   R-squared (Within):               0.1866
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.7587
Time:                        11:38:33   Log-likelihood                   -2187.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      240.21
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(4,4187)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             240.21
                            

In [17]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + UNEMP_RATE_PCT_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.1885
Estimator:                   PanelOLS   R-squared (Between):              0.5304
No. Observations:                4504   R-squared (Within):               0.1885
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.5272
Time:                        11:40:00   Log-likelihood                   -2182.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      243.18
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(4,4187)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             243.18
                            

In [18]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Total_HH_log \
                    + UNEMP_RATE_PCT_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.1885
Estimator:                   PanelOLS   R-squared (Between):              0.5272
No. Observations:                4504   R-squared (Within):               0.1885
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.5240
Time:                        11:40:49   Log-likelihood                   -2182.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      243.10
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(4,4187)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             243.10
                            

In [19]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + UNEMP_RATE_PCT_log \
                    + PCT_HH_NO_VEH_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.1886
Estimator:                   PanelOLS   R-squared (Between):              0.5301
No. Observations:                4499   R-squared (Within):               0.1886
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.5269
Time:                        11:41:43   Log-likelihood                   -2179.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      194.39
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.374   Distribution:                  F(5,4181)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             194.39
                            

In [22]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + UNEMP_RATE_PCT_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4857
Estimator:                   PanelOLS   R-squared (Between):              0.4763
No. Observations:                4504   R-squared (Within):               0.4857
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.4731
Time:                        11:43:38   Log-likelihood                   -1155.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      790.58
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(5,4186)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             790.58
                            

In [23]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4858
Estimator:                   PanelOLS   R-squared (Between):              0.8556
No. Observations:                4504   R-squared (Within):               0.4858
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.8521
Time:                        11:44:22   Log-likelihood                   -1155.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      791.01
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(5,4186)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             791.01
                            

In [27]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + TNC_FLAG \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4865
Estimator:                   PanelOLS   R-squared (Between):              0.7661
No. Observations:                4504   R-squared (Within):               0.4865
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.7623
Time:                        13:40:48   Log-likelihood                   -1151.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      660.95
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(6,4185)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             660.95
                            

In [26]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + YEARS_SINCE_TNC \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4873
Estimator:                   PanelOLS   R-squared (Between):              0.9507
No. Observations:                4504   R-squared (Within):               0.4873
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.9483
Time:                        13:40:21   Log-likelihood                   -1148.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      662.98
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(6,4185)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             662.98
                            

In [28]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + AVG_SPEED_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + YEARS_SINCE_TNC \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5042
Estimator:                   PanelOLS   R-squared (Between):              0.9259
No. Observations:                4504   R-squared (Within):               0.5042
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.9235
Time:                        13:41:30   Log-likelihood                   -1072.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      607.96
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(7,4184)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             607.96
                            

In [29]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + YEARS_SINCE_TNC \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4873
Estimator:                   PanelOLS   R-squared (Between):              0.9507
No. Observations:                4504   R-squared (Within):               0.4873
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.9483
Time:                        13:42:46   Log-likelihood                   -1148.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      662.98
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(6,4185)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             662.98
                            

In [31]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4879
Estimator:                   PanelOLS   R-squared (Between):              0.9593
No. Observations:                4504   R-squared (Within):               0.4879
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.9572
Time:                        13:44:46   Log-likelihood                   -1146.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      569.46
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(7,4184)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             569.46
                            